# Module 5: Calculating and visualizing 3D-mPDF patterns

3D-mPDF calculations (see https://doi.org/10.1107/S2052252518006590 and https://doi.org/10.1103/PhysRevB.100.144404) can be performed in diffpy.mpdf using the MPDF3Dcalculator class. Similar to the standard MPDFcalculator class, a MagStructure object is provided to the calculator, from which the positions and magnetic moments are extracted. Here, we use antiferromagnetic MnTe as an example. It is known to have an anisotropic correlation length in the correlated paramagnet regime above the ordering temperature, with a significantly longer correlation length along the c-axis than within the ab-plane (see https://doi.org/10.1016/j.matt.2022.03.011 for more details). This anisotropy makes it a useful example case for 3D-mPDF.

In [ ]:
import diffpy.mpdf as mpdf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

### Familiarize yourself with the magnetic structure of MnTe.
Go to MAGNDATA and look up MnTe. Use the interactive visualizer to get a feeling for the magnetic structure of MnTe. How would you describe it in words?

### Make the MagStructure

Load in the MnTe mCIF file to create a MagStructure instance with rmaxAtoms set to 30. Generate the spin positions and orientations.

Create an anisotropic correlation length using the dampingMat attribute. This is a 3x3 symmetric matrix for which the eigenvalues are equal to the inverse squared correlation length along the direction given by the corresponding eigenvector. See the appendix of https://doi.org/10.1107/S1600576722007257 for more details. 

In [ ]:
xi_ab = 4 # correlation length in the ab plane
xi_c = 8 # correlation length along c
mag.dampingMat = np.array([[1/xi_ab**2, 0, 0],
                           [0, 1/xi_ab**2, 0],
                           [0, 0, 1/xi_c**2]])

### Make the MPDF3Dcalculator

Now we create an instance of the MPDF3Dcalculator class and feed it our MagStructure object.

In [ ]:
mc3d = mpdf.MPDF3Dcalculator(magstruc=mag)
mc3d.gaussPeakWidth = 0.7 # sets the broadening of the peaks due to the spatial extent of the wavefunction

Here we are using a three-dimensional Gaussian to represent the spatial extent of wavefunction giving rise to the magnetic moment. The gaussPeakWidth attribute controls the broadening of the peaks and has a default value of 0.5. 

## Perform the calculation

The MPDF3Dcalculator.calc() method calls the primary computation method of the class. An optional argument of 'dr' can be used which will adjust the spatial resolution of the magnetic structure representation and the final mpdf. A larger dr value will decrease computaional costs but lower the resolution of the final mpdf calculated. As a note, because this computation is done in 3D, computation time and memory scale by dr<sup>3</sup>.

In [ ]:
mc3d.calc(dr=0.4) # dr=0.4 is a reasonable value

## Extract and visualize the results

The MPDF3Dcalculator.rgrid() method returns the real space grid used to compute the 3D-mPDF, while the MPDF3Dcalculator.mpdf attribute stores the calculated 3D-mPDF. Note that this is not defined until the computation is run.

In [ ]:
x,y,z = mc3d.rgrid()
mpdf3D = mc3d.mpdf

Now we visualize the results using the diffpy.mpdf.visualize3D module. This module generates a user-defined 2D slice of a 3D data set. We start by making an instance of the Visualizer class, which takes as input the 3D data set and the Cartesian x, y, and z arrays over which the data set is defined.

In [ ]:
import diffpy.mpdf.visualize3D as v3d
vis = v3d.Visualizer(mpdf3D, x, y, z)

Now we define the 2D slice we want to extract. Two methods can be used to accomplish this:
 - Method 1: Define a center point and a normal vector. The 2D slice will consist of the plane perpendicular to the normal vector centered on the given center point.
 - Method 2: Define three non-collinear points. The 2D slice will consist of the plane containing these three points, centered on the centroid of the three points.
 
We will use Method 1 first.

In [ ]:
len_horizontal = 50 # horizontal extent of the slice
len_vertical = 50 # vertical extent of the slice
stepsize = 0.2 # step size for the interpolation used to make the slice

# Let's first look in the ab plane, i.e. perpendicular to [0,0,1].
center = np.array([0,0,0])
normal = np.array([0,0,1])
vis.make_slice(len_horizontal, len_vertical, stepsize,
               use_normal = True, cen_pt=center, normal=normal)

# Now we just call the visualize() method
vis.visualize()

Explain how the figure above relates to the magnetic structure.

Now let's look in the bc plane.

In [ ]:
center = np.array([0,0,0])
normal = np.array([????]) # looking in the bc plane
vis.make_slice(len_horizontal, len_vertical, stepsize,
               use_normal = True, cen_pt=center, normal=normal)

# Now we just call the visualize() method
vis.visualize()

Interpret the figure in terms of the magnetic structure. How does the anisotropic correlation length that you defined manifest in the figure?

As a side note, you can also return the data slice for further manipulation or visualization by setting the returnSlice attribute to True:

In [ ]:
data_slice, xx, yy = vis.make_slice(len_horizontal, len_vertical, stepsize,
                                    use_normal = True, cen_pt=center, normal=normal,
                                    returnSlice=True)
print(data_slice.shape, xx.shape, yy.shape)

For completeness, let's now use Method 2 to generate a slice for visualization. We will choose three corners of the unit cell as our three points to define the plane. Note that in this case, this choice of points does not yield a particularly useful plane to look at, but it still demonstrates the functionality.

In [ ]:
lat = mag.struc.lattice
avec, bvec, cvec = lat.stdbase
vis.make_slice(len_horizontal, len_vertical, stepsize,
               use_normal=False,
               p1=avec, p2=bvec, p3=cvec)

# Now we just call the visualize() method
vis.visualize()

In [ ]:
plt.close('all')

### Exercises
 - Use Method 2 to recreate the view of the ab plane that we generated from Method 1.
 - Play around with the correlation lengths and gaussian peak width and observe the effect on the 3D-mPDF.